# SF Crime Classification Challenge

My approach for the kaggle's San Francisco Crime Classification challenge.

### Data exploration

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
training_data = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/train 2.csv')
training_data.tail(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607
878048,2003-01-06 00:01:00,FORGERY/COUNTERFEITING,"CHECKS, FORGERY (FELONY)",Monday,BAYVIEW,NONE,1800 Block of NEWCOMB AV,-122.394926,37.738212


In [3]:
testing_data = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/train 2.csv')
testing_data.tail(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607
878048,2003-01-06 00:01:00,FORGERY/COUNTERFEITING,"CHECKS, FORGERY (FELONY)",Monday,BAYVIEW,NONE,1800 Block of NEWCOMB AV,-122.394926,37.738212


In [4]:
training_data.shape, testing_data.shape

((878049, 9), (878049, 9))

In [5]:
training_data.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

In [6]:
testing_data.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

### Check missing values

In [7]:
training_data.apply(lambda x: sum(x.isnull()))

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

In [8]:
testing_data.apply(lambda x: sum(x.isnull()))

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

### Look at categories of all object variables:

In [9]:
columns = ['Dates','Category','Descript','DayOfWeek','PdDistrict','Resolution','Address','X','Y']
for column in columns:
    print ('\n+++ Frequency count for variable %s +++\n'%column)
    print (training_data[column].value_counts())


+++ Frequency count for variable Dates +++

2011-01-01 00:01:00    185
2006-01-01 00:01:00    136
2012-01-01 00:01:00     94
2006-01-01 12:00:00     63
2007-06-01 00:01:00     61
2006-06-01 00:01:00     58
2010-06-01 00:01:00     56
2010-08-01 00:01:00     55
2008-04-01 00:01:00     53
2013-11-01 00:01:00     52
2006-07-01 00:01:00     51
2008-11-01 00:01:00     51
2013-05-01 00:01:00     51
2010-11-01 00:01:00     51
2005-06-01 00:01:00     50
2011-06-01 00:01:00     50
2005-07-01 00:01:00     49
2008-06-01 00:01:00     48
2012-06-01 00:01:00     46
2013-06-01 12:00:00     46
2006-11-01 00:01:00     46
2009-09-01 00:01:00     46
2009-10-01 00:01:00     45
2009-04-01 00:01:00     45
2008-10-01 00:01:00     45
2007-07-01 00:01:00     44
2013-12-01 00:01:00     44
2003-10-01 00:01:00     44
2004-10-01 00:01:00     44
2003-06-01 00:01:00     43
                      ... 
2004-03-17 13:50:00      1
2015-02-22 22:42:00      1
2003-11-25 14:55:00      1
2009-01-23 13:45:00      1
2004-09-17

### Categorical values

#### Mining association rules
1. Create a new dataframe with only categorical values
2. Use FP growth to mine association rules

In [10]:
#Category 	Descript 	DayOfWeek 	PdDistrict 	Resolution 	Address

categorical_vals = training_data[['Descript','DayOfWeek','PdDistrict','Resolution','Address','Category']]
categorical_vals.head(2)


,Descript,DayOfWeek,PdDistrict,Resolution,Address,Category
0,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,WARRANTS
1,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,OTHER OFFENSES


In [12]:
import Orange as orange
import os
def convert_dataframe_to_orange(df):
    df.to_csv('_temp_.csv',index_label='id')
    orange_table = orange.data.Table('_temp_.csv')
    os.unlink('_temp_.csv')
    return orange_table

orange_table = convert_dataframe_to_orange(categorical_vals)

In [17]:
orange_table

[[0.000, WARRANT ARREST, Wednesday, NORTHERN, ARREST, BOOKED, ...] {OAK ST / LAGUNA ST},
 [1.000, TRAFFIC VIOLATION ARREST, Wednesday, NORTHERN, ARREST, BOOKED, ...] {OAK ST / LAGUNA ST},
 [2.000, TRAFFIC VIOLATION ARREST, Wednesday, NORTHERN, ARREST, BOOKED, ...] {VANNESS AV / GREENWICH ST},
 [3.000, GRAND THEFT FROM LOCKED AUTO, Wednesday, NORTHERN, NONE, ...] {1500 Block of LOMBARD ST},
 [4.000, GRAND THEFT FROM LOCKED AUTO, Wednesday, PARK, NONE, ...] {100 Block of BRODERICK ST},
 ...
]

In [24]:
from orangecontrib.associate.fpgrowth import *  
X, mapping = OneHot.encode(orange_table, include_class=True)

In [25]:
X

array([[False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [26]:
sorted(mapping.items())

[(0, (1, 0)),
 (1, (1, 1)),
 (2, (1, 2)),
 (3, (1, 3)),
 (4, (1, 4)),
 (5, (1, 5)),
 (6, (1, 6)),
 (7, (1, 7)),
 (8, (1, 8)),
 (9, (1, 9)),
 (10, (1, 10)),
 (11, (1, 11)),
 (12, (1, 12)),
 (13, (1, 13)),
 (14, (1, 14)),
 (15, (1, 15)),
 (16, (1, 16)),
 (17, (1, 17)),
 (18, (1, 18)),
 (19, (1, 19)),
 (20, (1, 20)),
 (21, (1, 21)),
 (22, (1, 22)),
 (23, (1, 23)),
 (24, (1, 24)),
 (25, (1, 25)),
 (26, (1, 26)),
 (27, (1, 27)),
 (28, (1, 28)),
 (29, (1, 29)),
 (30, (1, 30)),
 (31, (1, 31)),
 (32, (1, 32)),
 (33, (1, 33)),
 (34, (1, 34)),
 (35, (1, 35)),
 (36, (1, 36)),
 (37, (1, 37)),
 (38, (1, 38)),
 (39, (1, 39)),
 (40, (1, 40)),
 (41, (1, 41)),
 (42, (1, 42)),
 (43, (1, 43)),
 (44, (1, 44)),
 (45, (1, 45)),
 (46, (1, 46)),
 (47, (1, 47)),
 (48, (1, 48)),
 (49, (1, 49)),
 (50, (1, 50)),
 (51, (1, 51)),
 (52, (1, 52)),
 (53, (1, 53)),
 (54, (1, 54)),
 (55, (1, 55)),
 (56, (1, 56)),
 (57, (1, 57)),
 (58, (1, 58)),
 (59, (1, 59)),
 (60, (1, 60)),
 (61, (1, 61)),
 (62, (1, 62)),
 (63, (1, 63

In [51]:
itemsets = dict(frequent_itemsets(X, .9))

In [52]:
len(itemsets)

0

In [46]:
class_items = {item 
               for item, var, _ in OneHot.decode(mapping, orange_table, mapping) 
               if var is orange_table.domain.class_var}

In [47]:
sorted(class_items)

[]

In [48]:
orange_table.domain.class_var.values

AttributeError: 'NoneType' object has no attribute 'values'

In [53]:
rules = [(P, Q, supp, conf)
         for P, Q, supp, conf in association_rules(itemsets, .8)
         if len(Q) == 1 and Q & class_items]
len(rules)

0

In [54]:
rules

[]

### numerical values

### Model fiting & CV